In [1]:
import xarray as xr
import datetime as dt
from datetime import timedelta
from opendrift.readers import reader_netCDF_CF_generic
from opendrift.readers import reader_global_landmask
from opendrift.models.oceandrift import OceanDrift
import os
import matplotlib.pyplot as plt
import matplotlib
#matplotlib.use('TkAgg')

In [2]:
def make_map():
    fig = plt.figure(figsize=(12,9))
    cart_proj = ccrs.PlateCarree()
    ax = plt.axes(projection=cart_proj)
    ax.coastlines('10m', linewidth=0.8,zorder=200)
    ax.set_xlim(-123.3,-122.2)
    ax.set_ylim(37.5,38.1)
    return(fig, ax)

## Model Setup ##

__1. Create the Model Object__

We will be using the OceanDrift model, which simulates ocean surface drifting, forced by surface currents, wind, and/or stokes drift.

In [3]:
o = OceanDrift(loglevel=50) # Setting the log level will give the user different amounts of info (0=Debug (all), 20=Minimum, 50=None)
o_diffusion = OceanDrift(loglevel=50) # Setting the log level will give the user different amounts of info (0=Debug (all), 20=Minimum, 50=None)

__Create readers for forcing the model__

There are a couple of different types of readers. Here we will be using the `reader_netCDF_CF_generic` class and the `reader_global_landmask`.

__`reader_netCDF_CF_generic`__: Takes any CF-compliant netCDF file (including a thredds endpoint). Because the CF name for surface current velocities are: 'x_sea_water_velocity' and 'y_sea_water_velocity', the reader is able to identify the data.

__`reader_global_landmask`__ will create a landmask from the __Global Self-consistent, Hierarchical, High-resolution Geography Database (GSHHG)__ dataset.

In [4]:
# data_sets = ['Data/sfbay_2km_2020_07_20.nc','Data/sfbay_6km_2020_07_20.nc']
# reader_2km = reader_netCDF_CF_generic.Reader(data_sets[0])
# reader_6km = reader_netCDF_CF_generic.Reader(data_sets[1])
reader_2km = reader_netCDF_CF_generic.Reader("http://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/USWC/2km/hourly/RTV/HFRADAR_US_West_Coast_2km_Resolution_Hourly_RTV_best.ncd")
reader_6km = reader_netCDF_CF_generic.Reader("http://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/USWC/6km/hourly/RTV/HFRADAR_US_West_Coast_6km_Resolution_Hourly_RTV_best.ncd")




In [5]:
print(reader_6km)
# print the required variables for this model
print(OceanDrift.required_variables)

Reader: http://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/USWC/6km/hourly/RTV/HFRADAR_US_West_Coast_6km_Resolution_Hourly_RTV_best.ncd
Projection: 
  +proj=latlong
Coverage: [degrees]
  xmin: -130.360001   xmax: -115.805565   step: 0.0624695   numx: 233
  ymin: 30.250000   ymax: 49.992039   step: 0.0539398   numy: 367
  Corners (lon, lat):
    (-130.36,  49.99)  (-115.81,  49.99)
    (-130.36,  30.25)  (-115.81,  30.25)
Vertical levels [m]: 
  Not specified
Available time range:
  start: 2012-01-01 00:00:00   end: 2020-10-07 22:00:00   step: 1:00:00
    76871 times (1096 missing)
Variables:
  time
  forecast_reference_time
  forecast_period
  surface_eastward_sea_water_velocity
  surface_northward_sea_water_velocity
  x_sea_water_velocity
  y_sea_water_velocity

['x_sea_water_velocity', 'y_sea_water_velocity', 'x_wind', 'y_wind', 'upward_sea_water_velocity', 'ocean_vertical_diffusivity', 'sea_surface_wave_significant_height', 'sea_surface_wave_stokes_drift_x_velocity', 'sea_surface_wave_sto

In [6]:
reader_landmask = reader_global_landmask.Reader(
                       extent=[-123.99,  -122.177032, 37.244221, 38.233120])  # lonmin, lonmax, latmin, latmax

__3. Add readers to model__

Data readers can be added in an hierarchical manner, from highest to lowest prioity.

In [7]:
o.add_reader([reader_landmask,reader_2km, reader_6km])
o_diffusion.add_reader([reader_landmask,reader_2km, reader_6km])

__4. Seeding the Model__

There are a several different ways to seed data, we can supply a shape file, a radius, a line etc.

From the tutorial:
```
Note that the radius is not an absolute boundary within which elements will be seeded, but one standard deviation of a normal distribution in space. Thus about 68% of elements will be seeded within this radius, with more elements near the center. By default, elements are seeded at the surface (z=0)
```

Here we could also use the OilDrift model if we wanted to simulate oils (or other density fluids).

In [8]:
start_time = dt.datetime(2020,7,20,7,0)

o.seed_elements(lon=-122.55183, lat=37.8016, number=500, radius=1000,
                time=start_time)

o_diffusion.seed_elements(lon=-122.55183, lat=37.8016, number=500, radius=1000,
                time=start_time)

__5. Setting other configurations__

There are a bunch of ways to configure the model, how it reflects off of the shoreline, the advection schemes, other forcings etc.

Use `o.list_configspec()` to view the current settings

In [9]:
o.set_config('general:coastline_action', 'stranding') 
o.set_config('drift:scheme', 'runge-kutta4')
o.set_config('general:time_step_minutes', 15)
o.set_config('drift:stokes_drift', False)

o_diffusion.set_config('general:coastline_action', 'stranding') 
o_diffusion.set_config('drift:scheme', 'runge-kutta4')
o_diffusion.set_config('general:time_step_minutes', 15)
o_diffusion.set_config('drift:stokes_drift', False)
o_diffusion.set_config('drift:current_uncertainty_uniform', .5) # uncertainty .2 meters per sec. this is a uniform distribution from -.2 to .2
# o_diffusion.set_config('drift:current_uncertainty', .2) # uncertainty .2 meters per sec. - this is a normal distrobution from -.2 to .2

In [10]:
o_diffusion.list_environment_variables()

['land_binary_mask',
 'time',
 'forecast_reference_time',
 'forecast_period',
 'surface_eastward_sea_water_velocity',
 'surface_northward_sea_water_velocity',
 'x_sea_water_velocity',
 'y_sea_water_velocity',
 'time',
 'forecast_reference_time',
 'forecast_period',
 'surface_eastward_sea_water_velocity',
 'surface_northward_sea_water_velocity',
 'x_sea_water_velocity',
 'y_sea_water_velocity']

In [11]:
base_folder= "Data/model_output/"
out_fname = "sf_bay_" + start_time.strftime("%Y%m%dT%H%M%S")

In [12]:
os.path.join(base_folder,out_fname+".nc")

'Data/model_output/sf_bay_20200720T070000.nc'

In [14]:
# o.run(duration=timedelta(hours=48), time_step=timedelta(minutes=15)) #
o_diffusion.run(duration=timedelta(hours=48), time_step=timedelta(minutes=15), outfile=os.path.join(base_folder,out_fname+".nc")) #

In [19]:
o.animation(compare=o_diffusion, legend=['No diffusion', 'Width diffusion'],
             legend_loc='upper center', fast=True,  filename='Figures/GG_example-2km-start_density.mp4',show_trajectories=True, density=True)

  cmap.set_under('w')

[libx264 @ 0x5569d3795880] height not divisible by 2 (1100x761)
Error initializing output stream 0:0 -- Error while opening encoder for output stream #0:0 - maybe incorrect parameters such as bit_rate, rate, width or height



In [33]:
# o.plot(show_particles=True,background="x_sea_water_velocity")
# o.plot(compare=o_diffusion, legend=['Width diffusion', 'No diffusion'],density=True)
o.plot(density=True, filename='Figures/testing_plot.png')
plt.show()

In [32]:
print(o)

--------------------
Reader performance:
--------------------
global_landmask
 0:00:01.5  total
 0:00:00.0  preparing
 0:00:01.5  reading
 0:00:00.0  interpolation_time
 0:00:00.0  masking
--------------------
Data/sfbay_2km_2020_07_20.nc
 0:00:29.8  total
 0:00:00.4  preparing
 0:00:00.3  reading
 0:00:00.9  interpolation
 0:00:00.1  interpolation_time
 0:00:27.9  rotating vectors
 0:00:00.0  masking
--------------------
Data/sfbay_6km_2020_07_20.nc
--------------------
global_landmask_0
--------------------
http://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/USWC/2km/hourly/RTV/HFRADAR_US_West_Coast_2km_Resolution_Hourly_RTV_best.ncd
--------------------
http://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/USWC/6km/hourly/RTV/HFRADAR_US_West_Coast_6km_Resolution_Hourly_RTV_best.ncd
--------------------
Performance:
 9:20.7 total time
 9:08.5 configuration
    1.4 preparing main loop
      1.3 moving elements to ocean
     21.0 readers
        1.5 global_landmask
        1.3 postprocessing
   21.5 m

In [16]:
o_diffusion.write_netcdf_density_map('Data/model_output/diffusion_density_large.nc',pixelsize_m=1000)

In [ ]:
o_diffusion.

In [13]:
o.reset()
o_diffusion.reset()